In [8]:
import torch, pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import label_binarize

def load_dict_from_pickle(filename):
    with open(filename, 'rb') as file:
        loaded_dict = pickle.load(file)
    return loaded_dict


def load_data(file_path):
    # Load data
    X = torch.load(f'{file_path}/X.pt')
    Y = torch.load(f'{file_path}/Y.pt')
    
    # Check if X is a sparse matrix and convert to dense if necessary
    if isinstance(X, csr_matrix):
        X = X.toarray()  # Convert sparse to dense

    Y = transform_y_for_multi_class(Y)
    Nodes = load_dict_from_pickle(f'{file_path}/Nodes.pkl')

    # Subset to patient nodes (assuming node name starts with 'V')
    indeces = {p: i for i, p in enumerate(Nodes)}
    visit_indeces = [indeces[v] for v in Nodes if v[0] == 'V']

    X = X[visit_indeces]
    Y = Y[visit_indeces]

    return X, Y

# Transform Y for Multi-Class Classification
def transform_y_for_multi_class(Y):
    # Convert LoS into 4 classes:
    # 0 if LoS <= 1 (one day or less),
    # 1 if LoS > 1 and LoS <= 3 (three days),
    # 2 if LoS > 3 and LoS <= 7 (up to a week),
    # 3 if LoS > 7 (more than a week)
    return np.where(Y <= 1, 0,
                    np.where(Y <= 3, 1,
                             np.where(Y <= 7, 2, 3)))

# 1. Apply SMOTE
def apply_smote(X, Y):
    smote = SMOTE(random_state=42)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)
    return X_resampled, Y_resampled

# 2. Train Random Forest
def train_random_forest(X, Y, num_classes=4, num_estimators=100):
    # Split data into training, validation, and test sets
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=0.8, random_state=42)
    X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, train_size=0.5, random_state=42)

    # Apply SMOTE to the training set
    X_train_resampled, Y_train_resampled = apply_smote(X_train, Y_train)

    # Initialize the Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=num_estimators, random_state=42)

    # Train the model
    clf.fit(X_train_resampled, Y_train_resampled)

    # Evaluate on validation set
    Y_val_pred = clf.predict(X_val)
    val_metrics = evaluate_multi_class(Y_val, Y_val_pred, num_classes)

    print("Validation Metrics:")
    print(val_metrics)

    # Evaluate on test set
    Y_test_pred = clf.predict(X_test)
    test_metrics = evaluate_multi_class(Y_test, Y_test_pred, num_classes)

    print("Test Metrics:")
    print(test_metrics)

    return clf, test_metrics

# 3. Evaluation for Multi-Class Classification
def evaluate_multi_class(true_labels, preds, num_classes):
    # Binarize the true labels for multi-class AUC calculation
    true_labels_binarized = label_binarize(true_labels, classes=np.arange(num_classes))

    try:
        # Calculate AUC for each class and then macro-average
        auc = roc_auc_score(true_labels_binarized, label_binarize(preds, classes=np.arange(num_classes)), average='macro', multi_class='ovr')
    except ValueError:
        auc = None

    accuracy = accuracy_score(true_labels, preds)
    precision = precision_score(true_labels, preds, average='weighted')
    recall = recall_score(true_labels, preds, average='weighted')
    f1 = f1_score(true_labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc
    }

# Main Function
def main_random_forest(file_path, num_estimators=100):
    # Load data (replace with actual data loading)
    X, Y = load_data(file_path)

    # Train Random Forest
    clf, metrics = train_random_forest(X, Y, num_classes=4, num_estimators=num_estimators)
    
    return metrics

# Set parameters and run the main training process
saving_path = '../../Data/203_Diagnoses/infectious'
results = main_random_forest(saving_path, num_estimators=100)
print(results)


/tmp/ipykernel_1451326/3944624780.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load(f'{file_path}/X.pt')
/tmp/ipykernel_1451326/3944624780.py:18: FutureWar

ValueError: No support for np.int64 index based sparse matrices